Import All Neccessary Libraries

In [243]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Creating TwinCAT_Recording and Accelerometer_Recording Classes

In [246]:
class TwinCAT_Recording:
    def __init__(self,path):
        self.path = path
        
        self.filename = os.path.basename(path)[:-4]
        parts = self.filename.split("_")
        positions = parts[0].split("-")

        self.slide_position_one = positions[0]
        self.slide_position_two = positions[1]
        self.velocity = parts[1]
        self.slide_range = str(self.slide_position_one) + "-" + str(self.slide_position_two) + "mm"
        self.slide_start_position = float(self.slide_position_one) + 2.5

        self.data = pd.read_csv(self.path,sep='\t',skip_blank_lines=True,skiprows=4)
        self.data.rename(columns={
        'Name': 'Time (ms)'},inplace=True)

        idx = ((self.data['ActPos'].shift(1) < self.slide_start_position) & (self.data['ActPos'] >= self.slide_start_position) & (self.data['ActPos'].shift(-1) >= self.slide_start_position)).idxmax()
        self.data.drop(self.data.index[:idx],inplace=True)
        self.data['Time (ms)'] = self.data['Time (ms)'] - self.data['Time (ms)'].iloc[0]

        self.data['Velocity m/s'] = self.data['ActVelo'] / 1000
        self.data['Time (s)'] = self.data['Time (ms)'] / 1000
        self.data['Convolved Velocity m/s'] = np.convolve(self.data['Time (ms)'].values, self.data['ActVelo'].values)
        self.data['Acceleration m/s^2'] = np.gradient(self.data['Convolved Velocity m/s'],0.25e-3)
        





    
    def __repr__(self):
        return "This TwinCAT recording is at {velocity}mm/s from {position_one}mm to {position_two}mm along the slide. For Debugging {slide_start_position}".format(
            velocity=self.velocity,
            position_one=self.slide_position_one,
            position_two=self.slide_position_two,
            slide_start_position = self.slide_start_position,
            )


class Accelerometer_Recording:
        def __init__(self,path):
            self.path = path
            
            self.filename = os.path.basename(path)[:-8]
            parts = self.filename.split("_")
            positions = parts[0].split("-")

            self.slide_position_one = positions[0]
            self.slide_position_two = positions[1]
            self.velocity = parts[1]
            self.slide_range = str(self.slide_position_one) + "-" + str(self.slide_position_two) + "mm"

            with open(path,'r') as file:
                 first_line = file.readline().rstrip()

            if not first_line.endswith(','):
                 first_line += ','
            
            with open(path, 'r+') as file:
                 content = file.readlines()
                 content[0] = first_line + '\n'
                 file.seek(0)
                 file.writelines(content)

            self.data = pd.read_csv(path)
            self.data.rename(columns={self.data.columns[0]: 'Time h/m/s/ms', self.data.columns[1]: 'Device Name'}, inplace=True)
            self.data['Time h/m/s/ms'] = pd.to_datetime(self.data['Time h/m/s/ms'].str.strip(), format='%H:%M:%S.%f')
            self.data['Elapsed Time (s)'] = (self.data['Time h/m/s/ms'] - self.data.loc[0, 'Time h/m/s/ms']).dt.total_seconds()

            Acceleration_trigger = 0.5
            idx = ((self.data['Acceleration X(g)'].shift(1) < Acceleration_trigger) & (self.data['Acceleration X(g)'] >= Acceleration_trigger) & (self.data['Acceleration X(g)'].shift(-1) >= Acceleration_trigger)).idxmax()
            self.data.drop(self.data.index[:idx],inplace=True)
            self.data['Elapsed Time (s)'] = self.data['Elapsed Time (s)'] - self.data['Elapsed Time (s)'].iloc[0]
            self.data['Elapsed Time (ms)'] = self.data['Elapsed Time (s)'] * 1000


        def __repr__(self):
                return "This Accelerometer recording is at {velocity}mm/s from {position_one}mm to {position_two}mm along the slide.".format(
                    velocity=self.velocity,
                    position_one=self.slide_position_one,
                    position_two=self.slide_position_two
                    )


Creating an Object to Store all the TwinCAT Data from a specified folder

In [248]:
folder = r'G:\AUTOCAD\WOS\S19S PRT update\1-Trak\Beckhoff\Testing\2023-05-19 GFX Strain Gauge Slide testing\SG_Slide_Test_accelerometer_vs_twinCAT\TwinCAT_Data'

TwinCAT_recordings = []
slide_start_positions = []

for filename in os.listdir(folder):

    if filename.endswith('.csv'):

        filepath = os.path.join(folder, filename)
        TwinCAT_Data = TwinCAT_Recording(filepath)
        TwinCAT_recordings.append(TwinCAT_Data)

for i,test in enumerate(TwinCAT_recordings):
    print('Test {i}:    '.format(i=i),test)
    slide_start_positions.append(float(test.slide_position_one))

TwinCAT_recordings[0].data.head(2)


ValueError: Length of values (102637) does not match length of index (51319)

Creating an Object to Store all the Accelerometer Data from a specified folder

In [ ]:
folder = r'G:\AUTOCAD\WOS\S19S PRT update\1-Trak\Beckhoff\Testing\2023-05-19 GFX Strain Gauge Slide testing\SG_Slide_Test_accelerometer_vs_twinCAT\Accelerometer_Data'

Accelerometer_recordings = []
slide_start_positions = []

for filename in os.listdir(folder):

    if filename.endswith('.csv'):

        filepath = os.path.join(folder, filename)
        Accelerometer_Data = Accelerometer_Recording(filepath)
        Accelerometer_recordings.append(Accelerometer_Data)

for i,test in enumerate(Accelerometer_recordings):
    print('Test {i}:    '.format(i=i),test)
    slide_start_positions.append(float(test.slide_position_one))

Accelerometer_recordings[0].data.head(0)

Plot Showing The Set Current For the different Velocities (TWINCAT DATA)

In [ ]:
sns.set_style("darkgrid")
scaled = 2
plt.figure(figsize=(18*scaled,7*scaled))
TwinCAT_data_type = "Acceleration m/s^2"
Accelerometer_data_type = 'Acceleration X(g)'
time_start = 0
time_end = 3500

for i in range(len(TwinCAT_recordings)):
                
    plt.subplot(len(TwinCAT_recordings),1,i+1)
    sns.lineplot(data = TwinCAT_recordings[i].data,x="Time (ms)",y=TwinCAT_data_type,label=TwinCAT_recordings[i].slide_range)
    plt.xlim(time_start,time_end)
    plt.legend(loc='upper right')
    plt.title('{data_type} at {velocity}mm/s'.format(data_type= TwinCAT_data_type,velocity = TwinCAT_recordings[i].velocity))

Plot Showing The Linear Acceleration For the different Velocities (ACCELEROMETER DATA)

In [ ]:
sns.set_style("darkgrid")
scaled = 2
plt.figure(figsize=(18*scaled,7*scaled))
Accelerometer_data_type = 'Acceleration X(g)'
time_start = 0
time_end = 10000

for i in range(len(Accelerometer_recordings)):

    plt.subplot(len(Accelerometer_recordings),1,i+1)
    sns.scatterplot(data = Accelerometer_recordings[i].data,x="Elapsed Time (ms)",y=Accelerometer_data_type,label=Accelerometer_recordings[i].slide_range + ' X(g)')
    sns.lineplot(data = Accelerometer_recordings[i].data,x="Elapsed Time (ms)",y=Accelerometer_data_type)

    sns.scatterplot(data = Accelerometer_recordings[i].data,x="Elapsed Time (ms)",y=Accelerometer_data_type[:-4] + 'Y(g)',label=Accelerometer_recordings[i].slide_range + ' Y(g)')
    sns.lineplot(data = Accelerometer_recordings[i].data,x="Elapsed Time (ms)",y=Accelerometer_data_type[:-4] + 'Y(g)')
    
    sns.scatterplot(data = Accelerometer_recordings[i].data,x="Elapsed Time (ms)",y=Accelerometer_data_type[:-4] + 'Z(g)',label=Accelerometer_recordings[i].slide_range + ' Z(g)')
    sns.lineplot(data = Accelerometer_recordings[i].data,x="Elapsed Time (ms)",y=Accelerometer_data_type[:-4] + 'Z(g)')

    plt.xlim(time_start,time_end)
    plt.legend(loc='upper right')
    plt.title('{data_type} at {velocity}mm/s'.format(data_type= Accelerometer_data_type,velocity = Accelerometer_recordings[i].velocity))

In [ ]:
sns.set_style("darkgrid")
scaled = 2
plt.figure(figsize=(18*scaled,7*scaled))
Accelerometer_data_type = 'Acceleration X(g)'
time_start = 0
time_end = 3500


for i in range(len(TwinCAT_recordings)):
                
    plt.subplot(len(TwinCAT_recordings),1,i+1)
    sns.lineplot(data = TwinCAT_recordings[i].data,x="Time (ms)",y=TwinCAT_data_type,label=TwinCAT_recordings[i].slide_range)
    plt.xlim(time_start,time_end)
    plt.legend(loc='upper right')
    plt.title('{data_type} at {velocity}mm/s'.format(data_type= TwinCAT_data_type,velocity = TwinCAT_recordings[i].velocity))

for i in range(len(Accelerometer_recordings)):

    plt.subplot(len(Accelerometer_recordings),1,i+1)
    sns.scatterplot(data = Accelerometer_recordings[i].data,x="Elapsed Time (ms)",y=Accelerometer_data_type,label=Accelerometer_recordings[i].slide_range + ' X(g)')
    sns.lineplot(data = Accelerometer_recordings[i].data,x="Elapsed Time (ms)",y=Accelerometer_data_type)

    plt.xlim(time_start,time_end)
    plt.legend(loc='upper right')
    plt.title('{data_type} at {velocity}mm/s'.format(data_type= Accelerometer_data_type,velocity = Accelerometer_recordings[i].velocity))